In [2]:
!git clone https://github.com/nandanadileep/llama-7b-finetuning.git
%cd llama-7b-finetuning

Cloning into 'llama-7b-finetuning'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 77 (delta 40), reused 51 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 14.93 MiB | 27.70 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/kaggle/working/llama-7b-finetuning


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 30.3 MB/s eta 0:00:00


In [9]:
!ls /kaggle/working/llama-7b-finetuning/adapters/final_adapter

adapter_config.json	   special_tokens_map.json  tokenizer.json
adapter_model.safetensors  tokenizer_config.json    tokenizer.model


In [10]:
BASE_MODEL = "meta-llama/Llama-2-7b-chat-hf"
ADAPTER_PATH = "/kaggle/working/llama-7b-finetuning/adapters/final_adapter"

In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "meta-llama/Llama-2-7b-chat-hf"

# tokenizer (use adapter tokenizer first, fallback to base)
tokenizer = AutoTokenizer.from_pretrained(
    ADAPTER_PATH,
    trust_remote_code=True
)

# base model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

# load LoRA adapter
model = PeftModel.from_pretrained(
    model,
    ADAPTER_PATH
)

model.eval()


2026-01-02 11:58:30.026970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767355110.250869      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767355110.316236      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767355110.840972      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767355110.841017      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767355110.841020      55 computation_placer.cc:177] computation placer alr

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [16]:
def generate(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [17]:
print(
    generate(
        "### Instruction:\nGive me healthy diet tips\n\n### Response:"
    )
)


### Instruction:
Give me healthy diet tips

### Response:
Certainly! Here are some healthy diet tips:

1. Eat a variety of fruits and vegetables: Aim to include a variety of colors on your plate to ensure you are getting a range of nutrients.
2. Incorporate lean protein sources: Choose lean protein sources like chicken, fish, tofu, and legumes to help build and repair muscles.
3. Whole grains are a must: Choose whole grains like brown rice, quinoa, and whole wheat bread to increase fiber and nutrient intake.
4. Limit processed foods: Try to limit your intake of processed foods like sugary snacks, frozen meals, and packaged desserts.
5. Drink plenty of water: Aim to drink at least 8-10 glasses of water per day to stay hydrated.
6. Be mindful of portion sizes: Pay attention to the serving sizes of foods and beverages to avoid overeating or consuming too many calories.
7. Incorporate healthy fats: Choose healthy fats like avocado,


In [18]:
!pip install -q gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.2.19 which is incompatible.


In [20]:
import gradio as gr

def chat(prompt):
    return generate(prompt)

gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=5, label="Prompt"),
    outputs=gr.Markdown(),
    title="LoRA Fine-Tuned LLaMA Chat"
).launch(share=True)



* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://850112f696ac821ad4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio/flagged/dataset1.csv
